In [1]:
# -*- coding: utf-8 -*-

import sys,os
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy import linalg
from numpy import dot
import geomloss as gs

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions as D
from torchvision import datasets, transforms
import torchvision.utils as vutils
from torch.autograd import grad
import torch.utils.data
import torch.backends.cudnn as cudnn
from torch.nn.modules import Linear
from torch.autograd.functional import jacobian,hessian,vjp,vhp,hvp

import random
import math

FilePath = '../../'

file_list = ['GSM1599494_ES_d0_main.csv', 'GSM1599497_ES_d2_LIFminus.csv', 'GSM1599498_ES_d4_LIFminus.csv', 'GSM1599499_ES_d7_LIFminus.csv']

table_list = []
for filein in file_list:
    table_list.append(pd.read_csv(FilePath+filein, header=None))

matrix_list = []
gene_names = table_list[0].values[:,0]
for table in table_list:
    matrix_list.append(table.values[:,1:].astype('float32'))

cell_counts = [matrix.shape[1] for matrix in matrix_list]

def normalize_run(mat):
    rpm = np.sum(mat,0)/1e6
    detect_pr = np.sum(mat==0,0)/float(mat.shape[0])
    return np.log(mat*(np.median(detect_pr)/detect_pr)*1.0/rpm + 1.0)

norm_mat = [normalize_run(matrix) for matrix in matrix_list]

qt_mat = [np.percentile(norm_in,q=np.linspace(0,100,50),axis=1) for norm_in in norm_mat] 
wdiv=np.sum((qt_mat[0]-qt_mat[3])**2,0)
w_order = np.argsort(-wdiv)

wsub = w_order[0:100]


def nmf(X, latent_features, max_iter=100, error_limit=1e-6, fit_error_limit=1e-6, print_iter=200):
    """
    Decompose X to A*Y
    """
    eps = 1e-5
    print('Starting NMF decomposition with {} latent features and {} iterations.'.format(latent_features, max_iter))
    #X = X.toarray()   I am passing in a scipy sparse matrix

    # mask
    mask = np.sign(X)

    # initial matrices. A is random [0,1] and Y is A\X.
    rows, columns = X.shape
    A = np.random.rand(rows, latent_features)
    A = np.maximum(A, eps)

    Y = linalg.lstsq(A, X)[0]
    Y = np.maximum(Y, eps)

    masked_X = mask * X
    X_est_prev = dot(A, Y)
    for i in range(1, max_iter + 1):
        # ===== updates =====
        # Matlab: A=A.*(((W.*X)*Y')./((W.*(A*Y))*Y'));
        top = dot(masked_X, Y.T)
        bottom = (dot((mask * dot(A, Y)), Y.T)) + eps
        A *= top / bottom

        A = np.maximum(A, eps)
        # print 'A',  np.round(A, 2)

        # Matlab: Y=Y.*((A'*(W.*X))./(A'*(W.*(A*Y))));
        top = dot(A.T, masked_X)
        bottom = dot(A.T, mask * dot(A, Y)) + eps
        Y *= top / bottom
        Y = np.maximum(Y, eps)
        # print 'Y', np.round(Y, 2)


        # ==== evaluation ====
        if i % print_iter == 0 or i == 1 or i == max_iter:
            print('Iteration {}:'.format(i),)
            X_est = dot(A, Y)
            err = mask * (X_est_prev - X_est)
            fit_residual = np.sqrt(np.sum(err ** 2))
            X_est_prev = X_est

            curRes = linalg.norm(mask * (X - X_est), ord='fro')
            print('fit residual', np.round(fit_residual, 4),)
            print('total residual', np.round(curRes, 4))
            if curRes < error_limit or fit_residual < fit_error_limit:
                break
    return A, Y, dot(A,Y)

np.random.seed(0)
norm_imputed = [nmf(normin[wsub,:], latent_features = len(wsub)*4, max_iter=500)[2] for normin in norm_mat]

norm_adj = np.mean(norm_imputed[3],1)[:,np.newaxis]
subvec = np.array([0,1,2,3,4,5,6,7,8,9])

gnvec = gene_names[w_order[subvec]]

cov_mat = np.cov(norm_imputed[3][subvec,:])
whiten = np.diag(np.diag(cov_mat)**(-0.5))
unwhiten = np.diag(np.diag(cov_mat)**(0.5))

norm_imputed2 = [np.dot(whiten,(normin - norm_adj)[subvec,:]) for normin in norm_imputed]


class MLP(nn.Module):

    def __init__(self, dim_in, dim_out, dim_hidden=64, num_hidden=0, activation=nn.LeakyReLU()):
        super(MLP, self).__init__()

        if num_hidden == 0:
            self.linears = nn.ModuleList([nn.Linear(dim_in, dim_out)])
        elif num_hidden >= 1:
            self.linears = nn.ModuleList() 
            self.linears.append(nn.Linear(dim_in, dim_hidden))
            self.linears.extend([nn.Linear(dim_hidden, dim_hidden) for _ in range(num_hidden-1)])
            self.linears.append(nn.Linear(dim_hidden, dim_out))
        else:
            raise Exception('number of hidden layers must be positive')

        for m in self.linears:
            #nn.init.xavier_uniform_(m.weight)
            nn.init.xavier_normal_(m.weight)
            nn.init.uniform_(m.bias,a=-0.1,b=0.1)
            #nn.init.constant_(m.bias,0) 
 
        self.activation = activation 

    def forward(self, x):

        for m in self.linears[:-1]:
            x = self.activation(m(x))
            #x = F.dropout(x,p=0.5)

        return self.linears[-1](x)


def compute_gradient_penalty(D, real_sample, fake_sample,k,p):
    real_samples = real_sample.requires_grad_(True)
    fake_samples = fake_sample.requires_grad_(True)

    real_validity = D(real_samples)
    fake_validity = D(fake_samples)

    real_grad_out = torch.ones((real_samples.shape[0],1),dtype=torch.float32,requires_grad=False,device="cuda")
    real_grad = grad(
        real_validity, real_samples, real_grad_out, create_graph=True, retain_graph=True, only_inputs=True
    )[0]
    real_grad_norm = real_grad.view(real_grad.size(0), -1).pow(2).sum(1) ** (p / 2)

    fake_grad_out = torch.ones((fake_samples.shape[0],1),dtype=torch.float32,requires_grad=False,device="cuda")
    fake_grad = grad(
        fake_validity, fake_samples, fake_grad_out, create_graph=True, retain_graph=True, only_inputs=True
    )[0]
    fake_grad_norm = fake_grad.view(fake_grad.size(0), -1).pow(2).sum(1) ** (p / 2)

    return (torch.sum(real_grad_norm) + torch.sum(fake_grad_norm)) * k / (real_sample.shape[0]+fake_sample.shape[0])

class JumpEulerForwardCuda(nn.Module):
    def __init__(self,in_features,num_hidden,dim_hidden,step_size):
        super(JumpEulerForwardCuda,self).__init__()

        self.drift = MLP(in_features,in_features,dim_hidden,num_hidden)
        self.intensity = torch.tensor(intensity,device="cuda")
        self.mean = nn.Parameter(0.01*torch.ones(in_features))
        self.covHalf = nn.Parameter(0.08*torch.eye(in_features))
        self.diffusion = nn.Parameter(torch.ones(bd,10))
        self.in_features = in_features
        self.jump = MLP(in_features,in_features,dim_hidden,num_hidden)
        self.step_size = step_size

    def forward(self,z0,Nsim,steps):

        PopulationPath = torch.empty(size = (Nsim,steps+1,self.in_features),device="cuda")
        PopulationPath[:,0,:] = z0
        state = z0

        for i in range(1,steps+1):
            DP = D.poisson.Poisson(self.intensity*self.step_size)
            pois = DP.sample((Nsim,1)).cuda()
            state = state + self.drift(state)*self.step_size + math.sqrt(self.step_size)*torch.normal(0,1,size=(Nsim,bd),device="cuda")@self.diffusion+\
                (pois*self.mean + pois**(0.5)*torch.normal(0,1,size=(Nsim,self.in_features),device="cuda")@self.covHalf)*self.jump(state)
            PopulationPath[:,i,:] = state
        return PopulationPath


def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
sed = 200
setup_seed(sed)

a=gs.SamplesLoss(loss='sinkhorn',p=2,blur=0.01)


train_data = norm_imputed2

train0 = torch.tensor(train_data[0],dtype=torch.float32,requires_grad = True,device="cuda").t()
train2 = torch.tensor(train_data[1],dtype=torch.float32,requires_grad = True,device="cuda").t()
train4 = torch.tensor(train_data[2],dtype=torch.float32,requires_grad = True,device="cuda").t()
train7 = torch.tensor(train_data[3],dtype=torch.float32,requires_grad = True,device="cuda").t()

train0 = train0+0.01*torch.normal(0,1,size=train0.shape,device="cuda")
train2 = train2+0.01*torch.normal(0,1,size=train2.shape,device="cuda")
train4 = train4+0.01*torch.normal(0,1,size=train4.shape,device="cuda")
train7 = train7+0.01*torch.normal(0,1,size=train7.shape,device="cuda")


intensity = 10
lr = 0.0003
step_size = 0.03
kuan = 256
ceng = 4
bd = 2
n_critic = 3
k = 2
p = 6

n_sims = train0.shape[0]
in_features = train0.shape[1]
n_steps = [10,20,35]


netG = JumpEulerForwardCuda(10,ceng,kuan,step_size).cuda()
netD1 = MLP(10,1,dim_hidden=kuan,num_hidden=ceng).cuda()
netD2 = MLP(10,1,dim_hidden=kuan,num_hidden=ceng).cuda()
netD3 = MLP(10,1,dim_hidden=kuan,num_hidden=ceng).cuda()


optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerSD1 = optim.Adam(netD1.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerSD2 = optim.Adam(netD2.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerSD3 = optim.Adam(netD3.parameters(), lr=lr, betas=(0.5, 0.999))

n_epochs =  20000

wd = []
for epoch in range(n_epochs):
  

    for _ in range(n_critic):
        fake_data = netG(train0,n_sims,n_steps[2])
        fake1 = fake_data[:,n_steps[0],:]
        fake2 = fake_data[:,n_steps[1],:]
        fake3 = fake_data[:,n_steps[2],:]

        optimizerSD1.zero_grad()

        div_gp1 = compute_gradient_penalty(netD1,train2,fake1,k,p)
        d1_loss = -torch.mean(netD1(train2))+torch.mean(netD1(fake1))+div_gp1
        d1_loss.backward(retain_graph=True) # retain_graph=True

        optimizerSD1.step()


        optimizerSD2.zero_grad()
        
        div_gp2 = compute_gradient_penalty(netD2,train4,fake2,k,p)
        d2_loss = -torch.mean(netD2(train4))+torch.mean(netD2(fake2))+div_gp2
        d2_loss.backward(retain_graph=True)

        optimizerSD2.step()
        
        
        optimizerSD3.zero_grad()
        
        div_gp3 = compute_gradient_penalty(netD3,train7,fake3,k,p)
        d3_loss = -torch.mean(netD3(train7))+torch.mean(netD3(fake3))+div_gp3
        d3_loss.backward(retain_graph=True)

        optimizerSD3.step()
        
    
    for _ in range(1):
        optimizerG.zero_grad()
        fake_data = netG(train0,n_sims,n_steps[2])
        fake1 = fake_data[:,n_steps[0],:]
        fake2 = fake_data[:,n_steps[1],:]
        fake3 = fake_data[:,n_steps[2],:]
        g_loss = -torch.mean(netD1(fake1))-torch.mean(netD2(fake2))-torch.mean(netD3(fake3))
        g_loss.backward() 

        optimizerG.step()

    if epoch %10==0:
        x1 = a(fake_data[:,n_steps[0],:],train2).item()
        x2 = a(fake_data[:,n_steps[1],:],train4).item()
        x3 = a(fake_data[:,n_steps[2],:],train7).item()
        
        wd.append(x1+x2+x3)
        
        print("training error: ",x1," and ",x2," and ",x3)
        print("total error: ",wd[-1])

Starting NMF decomposition with 400 latent features and 500 iterations.
Iteration 1:
fit residual 4000.1233
total residual 138.7959
Iteration 200:
fit residual 123.6643
total residual 31.8464
Iteration 400:
fit residual 21.7894
total residual 12.6115
Iteration 500:
fit residual 4.3442
total residual 8.6686
Starting NMF decomposition with 400 latent features and 500 iterations.
Iteration 1:
fit residual 2257.8358
total residual 58.2265
Iteration 200:
fit residual 56.8198
total residual 4.0447
Iteration 400:
fit residual 3.4777
total residual 0.6841
Iteration 500:
fit residual 0.3744
total residual 0.3185
Starting NMF decomposition with 400 latent features and 500 iterations.
Iteration 1:
fit residual 2925.3664
total residual 80.2657
Iteration 200:
fit residual 78.1148
total residual 6.1559
Iteration 400:
fit residual 5.1661
total residual 1.1917
Iteration 500:
fit residual 0.6183
total residual 0.5917
Starting NMF decomposition with 400 latent features and 500 iterations.
Iteration 1:
f

training error:  1.6754486560821533  and  2.153059959411621  and  1.6871862411499023
total error:  5.515694856643677
training error:  1.6624603271484375  and  3.046994209289551  and  2.043606996536255
total error:  6.753061532974243
training error:  1.6892871856689453  and  3.1002635955810547  and  1.899970531463623
total error:  6.689521312713623
training error:  1.6894574165344238  and  2.7729454040527344  and  2.2279887199401855
total error:  6.690391540527344
training error:  1.7647335529327393  and  2.6678004264831543  and  1.770745873451233
total error:  6.2032798528671265
training error:  1.6796921491622925  and  2.9960856437683105  and  1.8817791938781738
total error:  6.557556986808777
training error:  1.714460849761963  and  3.060393810272217  and  1.6637136936187744
total error:  6.438568353652954
training error:  1.6232202053070068  and  2.6153311729431152  and  1.8693288564682007
total error:  6.107880234718323
training error:  1.662695050239563  and  2.477398157119751  an

training error:  1.2628190517425537  and  1.9020448923110962  and  1.5356340408325195
total error:  4.700497984886169
training error:  1.2569851875305176  and  1.867966890335083  and  1.5361485481262207
total error:  4.661100625991821
training error:  1.288804292678833  and  2.185131788253784  and  1.5550363063812256
total error:  5.028972387313843
training error:  1.2814550399780273  and  2.1346936225891113  and  1.611405372619629
total error:  5.027554035186768
training error:  1.2846368551254272  and  2.3056764602661133  and  1.430795431137085
total error:  5.0211087465286255
training error:  1.276003122329712  and  2.1782732009887695  and  1.523726463317871
total error:  4.9780027866363525
training error:  1.3605105876922607  and  2.7569808959960938  and  1.4932591915130615
total error:  5.610750675201416
training error:  1.290886640548706  and  3.019918918609619  and  1.7065062522888184
total error:  6.0173118114471436
training error:  1.3054978847503662  and  2.5352983474731445  

training error:  1.1695911884307861  and  1.6925134658813477  and  1.3058351278305054
total error:  4.167939782142639
training error:  1.1502621173858643  and  1.7651057243347168  and  1.3070567846298218
total error:  4.222424626350403
training error:  1.1801626682281494  and  1.8021478652954102  and  1.3474550247192383
total error:  4.329765558242798
training error:  1.1965818405151367  and  1.7112059593200684  and  1.3192580938339233
total error:  4.227045893669128
training error:  1.164451003074646  and  1.6757787466049194  and  1.325300931930542
total error:  4.165530681610107
training error:  1.1668429374694824  and  1.8111616373062134  and  1.4621843099594116
total error:  4.440188884735107
training error:  1.1670546531677246  and  1.730797529220581  and  1.315794825553894
total error:  4.2136470079422
training error:  1.1618108749389648  and  1.720443844795227  and  1.3132541179656982
total error:  4.19550883769989
training error:  1.1579790115356445  and  1.701810359954834  and

training error:  1.1098450422286987  and  1.6112385988235474  and  1.3919973373413086
total error:  4.113080978393555
training error:  1.117152214050293  and  1.5554172992706299  and  1.220350980758667
total error:  3.89292049407959
training error:  1.0991710424423218  and  1.6066737174987793  and  1.2927149534225464
total error:  3.9985597133636475
training error:  1.086097002029419  and  1.681613564491272  and  1.2633841037750244
total error:  4.031094670295715
training error:  1.1161448955535889  and  1.599325180053711  and  1.2905482053756714
total error:  4.006018280982971
training error:  1.097308874130249  and  1.5849332809448242  and  1.257698893547058
total error:  3.9399410486221313
training error:  1.0971806049346924  and  1.5627423524856567  and  1.2240736484527588
total error:  3.883996605873108
training error:  1.0886482000350952  and  1.5613977909088135  and  1.2240285873413086
total error:  3.8740745782852173
training error:  1.1262099742889404  and  1.5627999305725098 

...

training error:  0.590773344039917  and  1.0679402351379395  and  0.9013017416000366
total error:  2.560015320777893
training error:  0.5844342112541199  and  1.2073030471801758  and  0.962181806564331
total error:  2.7539190649986267
training error:  0.5687960386276245  and  1.0959392786026  and  1.0307977199554443
total error:  2.695533037185669
training error:  0.5851587653160095  and  1.0840516090393066  and  0.8800610303878784
total error:  2.5492714047431946
training error:  0.6344858407974243  and  1.683504581451416  and  1.024016261100769
total error:  3.3420066833496094
training error:  0.6080040335655212  and  1.0964322090148926  and  0.9907436370849609
total error:  2.6951798796653748
training error:  0.6123466491699219  and  1.1005594730377197  and  0.9320604205131531
total error:  2.6449665427207947
training error:  0.5832064151763916  and  1.0985394716262817  and  0.9107155799865723
total error:  2.5924614667892456
training error:  0.6010260581970215  and  1.0786491632461

training error:  0.5843263268470764  and  1.070948600769043  and  0.8959866762161255
total error:  2.551261603832245
training error:  0.6053873300552368  and  1.0791934728622437  and  0.8962466716766357
total error:  2.580827474594116
training error:  0.6082632541656494  and  1.1611309051513672  and  0.931553065776825
total error:  2.7009472250938416
training error:  0.5819931030273438  and  1.0872492790222168  and  0.9279971122741699
total error:  2.5972394943237305
training error:  0.6512244939804077  and  1.3085980415344238  and  1.049906611442566
total error:  3.0097291469573975
training error:  0.5941983461380005  and  1.07937490940094  and  1.0505716800689697
total error:  2.72414493560791
training error:  0.6145918369293213  and  1.120802402496338  and  0.980650782585144
total error:  2.7160450220108032
training error:  0.6379356384277344  and  1.661158800125122  and  1.0834890604019165
total error:  3.382583498954773
training error:  0.6112608909606934  and  1.1401499509811401 

training error:  0.6149425506591797  and  1.222476840019226  and  0.8993515968322754
total error:  2.736770987510681
training error:  0.5943453907966614  and  1.1529688835144043  and  1.0544289350509644
total error:  2.80174320936203
training error:  0.605589747428894  and  1.0661239624023438  and  0.8701940774917603
total error:  2.541907787322998
training error:  0.578535795211792  and  1.0792478322982788  and  0.862877368927002
total error:  2.5206609964370728
training error:  0.5929731130599976  and  1.1002919673919678  and  0.9969443082809448
total error:  2.69020938873291
training error:  0.5867084860801697  and  1.1044914722442627  and  0.8831661939620972
total error:  2.5743661522865295
training error:  0.5832327604293823  and  1.089476466178894  and  0.9030553698539734
total error:  2.5757645964622498
training error:  0.5964058637619019  and  1.223304271697998  and  0.9159661531448364
total error:  2.7356762886047363
training error:  0.5807750225067139  and  1.099353313446045 

training error:  0.5691409707069397  and  1.0899503231048584  and  0.9325540065765381
total error:  2.591645300388336
training error:  0.5640130043029785  and  1.0682308673858643  and  0.9696282148361206
total error:  2.6018720865249634
training error:  0.5611695051193237  and  1.0765448808670044  and  0.9462345838546753
total error:  2.5839489698410034
training error:  0.5938529968261719  and  1.1145788431167603  and  0.8866311311721802
total error:  2.5950629711151123
training error:  0.5410531759262085  and  1.0432841777801514  and  0.8847142457962036
total error:  2.4690515995025635
training error:  0.5932449102401733  and  1.1334179639816284  and  0.815707802772522
total error:  2.5423706769943237
training error:  0.5658793449401855  and  1.1663259267807007  and  0.9260292649269104
total error:  2.6582345366477966
training error:  0.5632228255271912  and  1.096134901046753  and  0.8992069959640503
total error:  2.5585647225379944
training error:  0.5478334426879883  and  1.0408061

training error:  0.5747042894363403  and  1.08788001537323  and  0.8941582441329956
total error:  2.556742548942566
training error:  0.5724764466285706  and  1.0812737941741943  and  0.8877294659614563
total error:  2.541479706764221
training error:  0.5449273586273193  and  1.092639446258545  and  0.8873257040977478
total error:  2.524892508983612
training error:  0.5314903259277344  and  1.0682547092437744  and  0.8884127140045166
total error:  2.4881577491760254
training error:  0.5776450634002686  and  1.1643296480178833  and  0.8928090333938599
total error:  2.6347837448120117
training error:  0.5427354574203491  and  1.080774188041687  and  0.8571670055389404
total error:  2.4806766510009766
training error:  0.5484751462936401  and  1.104699730873108  and  0.8764665126800537
total error:  2.5296413898468018
training error:  0.598063588142395  and  1.1469672918319702  and  0.9141020178794861
total error:  2.6591328978538513
training error:  0.6297982335090637  and  1.0761170387268

In [2]:
torch.save(netG.state_dict(),"./epsilon0.01.pt")